# 1 segformer

In [1]:
from transformers import SegformerForSemanticSegmentation

# Load the pretrained Segformer model
segformer_ = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/mit-b3",
    num_labels=5
)

# Extract the encoder
encoder = segformer_.segformer.encoder

# Example usage: pass inputs to the encoder
from transformers import SegformerFeatureExtractor
import torch

# Load a feature extractor (for preprocessing images)
feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b3")

# Example image tensor (replace with actual preprocessed image data)
dummy_image = torch.rand(1, 3, 512, 512)  # Batch of 1, 3 channels, 512x512 resolution

# Pass through encoder
encoder_outputs = encoder(dummy_image)
encoder_hidden_state = encoder_outputs['last_hidden_state']

print(encoder_hidden_state.shape)

/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b3 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.runnin

torch.Size([1, 512, 16, 16])


/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/home/eric/anaconda3/envs/trex/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [2]:
segformer_.config

SegformerConfig {
  "_name_or_path": "nvidia/mit-b3",
  "architectures": [
    "SegformerForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout_prob": 0.1,
  "decoder_hidden_size": 768,
  "depths": [
    3,
    4,
    18,
    3
  ],
  "downsampling_rates": [
    1,
    4,
    8,
    16
  ],
  "drop_path_rate": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_sizes": [
    64,
    128,
    320,
    512
  ],
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-06,
  "mlp_ratios": [
    4,
    4,
    4,
    4
  ],
  "model_type": "segformer",
  "num_attention_heads": [
    1,
    2,
    5,
    8
  ],
  "num_channels": 3,
  "num_encoder_blocks": 4,
  "patch_sizes": [
    7,
    3,
    3,
  

In [ ]:
#---- 수정 
segformer_.config.num_encoder_blocks = 5
 

In [ ]:
segformer_.config.hidden_sizes

[64, 128, 320, 512]

In [ ]:
segformer_.config.hidden_sizes.append(1024)

In [9]:
segformer_.config.hidden_sizes

[64, 128, 320, 512, 1024]

In [14]:
import math
math.sqrt(512)

22.627416997969522

In [43]:
segformer_.config.output_hidden_states = True

In [44]:
outputs_ = segformer_(dummy_image)

In [65]:
outputs_.keys()

odict_keys(['logits', 'hidden_states'])

In [66]:
outputs_.logits.shape

torch.Size([1, 5, 128, 128])

In [67]:
outputs_.hidden_states[0].shape

torch.Size([1, 64, 128, 128])

In [68]:
outputs_.hidden_states[1].shape

torch.Size([1, 128, 64, 64])

In [69]:
outputs_.hidden_states[2].shape

torch.Size([1, 320, 32, 32])

In [70]:
outputs_.hidden_states[3].shape

torch.Size([1, 512, 16, 16])

In [59]:
last_out = segformer_.decode_head(outputs_.hidden_states)

In [ ]:
last_out.shape

torch.Size([1, 5, 128, 128])

In [62]:
segformer_.decode_head

SegformerDecodeHead(
  (linear_c): ModuleList(
    (0): SegformerMLP(
      (proj): Linear(in_features=64, out_features=768, bias=True)
    )
    (1): SegformerMLP(
      (proj): Linear(in_features=128, out_features=768, bias=True)
    )
    (2): SegformerMLP(
      (proj): Linear(in_features=320, out_features=768, bias=True)
    )
    (3): SegformerMLP(
      (proj): Linear(in_features=512, out_features=768, bias=True)
    )
  )
  (linear_fuse): Conv2d(3072, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batch_norm): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Conv2d(768, 5, kernel_size=(1, 1), stride=(1, 1))
)

In [77]:
segformer_.decode_head

SegformerDecodeHead(
  (linear_c): ModuleList(
    (0): SegformerMLP(
      (proj): Linear(in_features=64, out_features=768, bias=True)
    )
    (1): SegformerMLP(
      (proj): Linear(in_features=128, out_features=768, bias=True)
    )
    (2): SegformerMLP(
      (proj): Linear(in_features=320, out_features=768, bias=True)
    )
    (3): SegformerMLP(
      (proj): Linear(in_features=512, out_features=768, bias=True)
    )
  )
  (linear_fuse): Conv2d(3072, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batch_norm): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Conv2d(768, 5, kernel_size=(1, 1), stride=(1, 1))
)

In [36]:
outputs_ = segformer_.segformer.encoder(dummy_image)

In [42]:
segformer_.decode_head

SegformerDecodeHead(
  (linear_c): ModuleList(
    (0): SegformerMLP(
      (proj): Linear(in_features=64, out_features=768, bias=True)
    )
    (1): SegformerMLP(
      (proj): Linear(in_features=128, out_features=768, bias=True)
    )
    (2): SegformerMLP(
      (proj): Linear(in_features=320, out_features=768, bias=True)
    )
    (3): SegformerMLP(
      (proj): Linear(in_features=512, out_features=768, bias=True)
    )
  )
  (linear_fuse): Conv2d(3072, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batch_norm): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Conv2d(768, 5, kernel_size=(1, 1), stride=(1, 1))
)

In [35]:
segformer_.segformer.encoder.block[0]

ModuleList(
  (0): SegformerLayer(
    (layer_norm_1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (attention): SegformerAttention(
      (self): SegformerEfficientSelfAttention(
        (query): Linear(in_features=64, out_features=64, bias=True)
        (key): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
        (sr): Conv2d(64, 64, kernel_size=(8, 8), stride=(8, 8))
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (output): SegformerSelfOutput(
        (dense): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (drop_path): Identity()
    (layer_norm_2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (mlp): SegformerMixFFN(
      (dense1): Linear(in_features=64, out_features=256, bias=True)
      (dwconv): SegformerDWConv(
        (dwc

In [ ]:
segformer_.segformer.encoder.block

ModuleList(
  (0): SegformerLayer(
    (layer_norm_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (attention): SegformerAttention(
      (self): SegformerEfficientSelfAttention(
        (query): Linear(in_features=512, out_features=512, bias=True)
        (key): Linear(in_features=512, out_features=512, bias=True)
        (value): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (output): SegformerSelfOutput(
        (dense): Linear(in_features=512, out_features=512, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (drop_path): SegformerDropPath(p=0.09259259700775146)
    (layer_norm_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (mlp): SegformerMixFFN(
      (dense1): Linear(in_features=512, out_features=2048, bias=True)
      (dwconv): SegformerDWConv(
        (dwconv): Conv2d(2048, 2048, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2048)
      )


In [3]:
encoder_outputs.keys()

odict_keys(['last_hidden_state'])

In [4]:
encoder_outputs["last_hidden_state"].shape

torch.Size([1, 512, 16, 16])

In [8]:
dummy_hidden = torch.rand(1, 512, 16, 16)  

In [9]:
segformer_.decode_head.forward(dummy_hidden)

IndexError: tuple index out of range

In [5]:
segformer_.decode_head.forward

<bound method SegformerDecodeHead.forward of SegformerDecodeHead(
  (linear_c): ModuleList(
    (0): SegformerMLP(
      (proj): Linear(in_features=64, out_features=768, bias=True)
    )
    (1): SegformerMLP(
      (proj): Linear(in_features=128, out_features=768, bias=True)
    )
    (2): SegformerMLP(
      (proj): Linear(in_features=320, out_features=768, bias=True)
    )
    (3): SegformerMLP(
      (proj): Linear(in_features=512, out_features=768, bias=True)
    )
  )
  (linear_fuse): Conv2d(3072, 768, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (batch_norm): BatchNorm2d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (classifier): Conv2d(768, 5, kernel_size=(1, 1), stride=(1, 1))
)>

In [ ]:
print(encoder_hidden_state.shape)

torch.Size([1, 512, 16, 16])


In [6]:
feature_extractor

SegformerFeatureExtractor {
  "do_normalize": true,
  "do_reduce_labels": false,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "SegformerFeatureExtractor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 512,
    "width": 512
  }
}

# 2 Dino v2

In [ ]:
#--- 
from transformers import AutoImageProcessor, AutoModel
# Load the processor and model
processor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')


Using cache found in /home/eric/.cache/torch/hub/facebookresearch_dinov2_main
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/eric/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [11]:
import torch
from transformers import AutoImageProcessor, AutoModel
from PIL import Image
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
image = Image.open(requests.get(url, stream=True).raw)

processor = AutoImageProcessor.from_pretrained('facebook/dinov2-large')
model = AutoModel.from_pretrained('facebook/dinov2-large')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs[0]

print("#---")
print(last_hidden_states.shape)

#---
torch.Size([1, 257, 1024])


In [12]:
processor

BitImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "BitImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 256
  }
}

In [15]:
dummy_image = torch.rand(1, 3, 256, 256)  # Batch of 1, 3 channels, 512x512 resolution
encoder_out = model(dummy_image)

In [16]:
encoder_out[0].shape

torch.Size([1, 325, 1024])